In [1]:
import glworia.load_interp
from glworia.load_interp import *

In [2]:
from importlib import reload

In [3]:
reload(glworia.load_interp)
from glworia.load_interp import *

In [4]:
settings = {
    'y_low': 0.1,
    'y_high': 5.0,
    'kappa_low': 0.1,
    'kappa_high': 10.0,
    'N_grid': 30,
    'N_grid_strong': 60,
    'N_crit': 60,
    'N': 200,
    'lens_model_name': 'NFW',

    'T0_max': 1000,

    'y_low_im': 0.1,
    'y_high_im': 5.0,
    'kappa_low_im': 0.1,
    'kappa_high_im': 10.0,
    'N_grid_im': 1000,
    'N_crit_im': 1000,
}

In [5]:
interpolators = load_interpolators('./interpolation', **settings)

In [6]:
w_interp = np.linspace(0.001, 1e4, 10**4)

In [7]:
y_interp = 1.0
kappa_interp = 3.0

In [8]:
F = F_interp(w_interp, y_interp, kappa_interp, interpolators, settings)

/home/mcheung1/Lensing/Glworia_jax/glworia_jax/glworia/load_interp.py:186: RuntimeWarning: divide by zero encountered in divide
  return (1 + np.tanh((x - x0)/a))/2


In [9]:
import matplotlib.pyplot as plt